In [2]:
# !pip install pdf2image
# !pip install datasets
# !sudo apt-get install poppler-utils
# !pip install pytesseract
# !apt install tesseract-ocr

In [ ]:
import os
import pandas as pd
from pdf2image import convert_from_path

In [53]:
import os
import pandas as pd
from pdf2image import convert_from_path

def convert_pdf_to_images(pdf_path, output_directory):
    images = convert_from_path(pdf_path)
    image_paths = []
    for i, image in enumerate(images):
        image_path = os.path.join(output_directory, f'{os.path.basename(pdf_path)}_{i}.jpg')
        image.save(image_path, 'JPEG')
        image_paths.append(image_path)
        break
    return image_paths

def list_images_and_categories(root_dir, extensions=['.png', '.jpg']):
    image_paths = []
    categories = []
    #print(root_dir)
    tree_split = root_dir.split("/")[-1]

    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.pdf'):
                pdf_path = os.path.join(root, file)
                #print(pdf_path)
                output_directory = os.path.splitext(pdf_path)[0]  # Use PDF filename as output directory
                if not os.path.exists(output_directory):
                    os.makedirs(output_directory)
                converted_image_paths = convert_pdf_to_images(pdf_path, output_directory)
                #category = converted_image_paths.split(tree_split)[-1]
                image_paths.extend(converted_image_paths)
                category = os.path.basename(root)
                #print(converted_image_paths[0])
                category = converted_image_paths[0].split(tree_split)[-1].strip("/")
                category = category.split("/")[0]
                #print(category)
                categories.extend([category] * len(converted_image_paths))


    if not image_paths:  # If no JPG or PNG files were found
        raise ValueError("No image files found in the directory.")

    return pd.DataFrame({'image_path': image_paths, 'label': categories})

# Set your root directory where images are stored
root_directory = '/content/drive/MyDrive/layout_dataset_multi_dir'
#root_directory = '/content/drive/MyDrive/layout_dataset'

# Create a DataFrame with image paths and categories
data = list_images_and_categories(root_directory)

data



,image_path,label
0,/content/drive/MyDrive/layout_dataset_multi_di...,cv
1,/content/drive/MyDrive/layout_dataset_multi_di...,cv
2,/content/drive/MyDrive/layout_dataset_multi_di...,cv
3,/content/drive/MyDrive/layout_dataset_multi_di...,cv
4,/content/drive/MyDrive/layout_dataset_multi_di...,invoive
5,/content/drive/MyDrive/layout_dataset_multi_di...,invoive
6,/content/drive/MyDrive/layout_dataset_multi_di...,invoive
7,/content/drive/MyDrive/layout_dataset_multi_di...,invoive
8,/content/drive/MyDrive/layout_dataset_multi_di...,invoive


In [51]:
for i in range(len(data)):
    print(i,data["image_path"][i],data["label"][i])

0 /content/drive/MyDrive/layout_dataset_multi_dir/cv/cat1/kp/kp.pdf_0.jpg cv
1 /content/drive/MyDrive/layout_dataset_multi_dir/cv/cat1/SUBBARAOGOGULAMUDI 4Y_6M/SUBBARAOGOGULAMUDI 4Y_6M.pdf_0.jpg cv
2 /content/drive/MyDrive/layout_dataset_multi_dir/cv/cat2/Venu DS_pf/Venu DS_pf.pdf_0.jpg cv
3 /content/drive/MyDrive/layout_dataset_multi_dir/cv/cat2/Pammi Venumadhav DS cv/Pammi Venumadhav DS cv.pdf_0.jpg cv
4 /content/drive/MyDrive/layout_dataset_multi_dir/invoive/cat3/OD120824647596905000/OD120824647596905000.pdf_0.jpg invoive
5 /content/drive/MyDrive/layout_dataset_multi_dir/invoive/cat3/OD122113790943409000/OD122113790943409000.pdf_0.jpg invoive
6 /content/drive/MyDrive/layout_dataset_multi_dir/invoive/cat4/OD123000879013753000/OD123000879013753000.pdf_0.jpg invoive
7 /content/drive/MyDrive/layout_dataset_multi_dir/invoive/cat4/OD123018989184857000/OD123018989184857000.pdf_0.jpg invoive
8 /content/drive/MyDrive/layout_dataset_multi_dir/invoive/cat4/OD123016624004910000/OD12301662400491

In [52]:
needed_labels = ["cv", "invoive"]

data = data[(data['label'] == 'cv') | (data['label'] == 'invoive')]
data

,image_path,label
0,/content/drive/MyDrive/layout_dataset_multi_di...,cv
1,/content/drive/MyDrive/layout_dataset_multi_di...,cv
2,/content/drive/MyDrive/layout_dataset_multi_di...,cv
3,/content/drive/MyDrive/layout_dataset_multi_di...,cv
4,/content/drive/MyDrive/layout_dataset_multi_di...,invoive
5,/content/drive/MyDrive/layout_dataset_multi_di...,invoive
6,/content/drive/MyDrive/layout_dataset_multi_di...,invoive
7,/content/drive/MyDrive/layout_dataset_multi_di...,invoive
8,/content/drive/MyDrive/layout_dataset_multi_di...,invoive


In [ ]:
labels = [label for label in set(data["label"].unique())]
idx2label = {v: k for v, k in enumerate(labels)}
label2idx = {k: v for v, k in enumerate(labels)}
label2idx

{'cv': 0, 'invoive': 1}

In [ ]:
from datasets import Dataset
from PIL import Image, ImageDraw, ImageFont
import pytesseract
import numpy as np

def normalize_box(box, width, height):
     return [
         int(1000 * (box[0] / width)),
         int(1000 * (box[1] / height)),
         int(1000 * (box[2] / width)),
         int(1000 * (box[3] / height)),
     ]

def apply_ocr(example):
        # get the image
        image = Image.open(example['image_path'])

        width, height = image.size

        # apply ocr to the image
        ocr_df = pytesseract.image_to_data(image, output_type='data.frame')
        float_cols = ocr_df.select_dtypes('float').columns
        ocr_df = ocr_df.dropna().reset_index(drop=True)
        ocr_df[float_cols] = ocr_df[float_cols].round(0).astype(int)
        ocr_df = ocr_df.replace(r'^\s*$', np.nan, regex=True)
        ocr_df = ocr_df.dropna().reset_index(drop=True)

        # get the words and actual (unnormalized) bounding boxes
        #words = [word for word in ocr_df.text if str(word) != 'nan'])
        words = list(ocr_df.text)
        words = [str(w) for w in words]
        coordinates = ocr_df[['left', 'top', 'width', 'height']]
        actual_boxes = []
        for idx, row in coordinates.iterrows():
            x, y, w, h = tuple(row) # the row comes in (left, top, width, height) format
            actual_box = [x, y, x+w, y+h] # we turn it into (left, top, left+width, top+height) to get the actual box
            actual_boxes.append(actual_box)

        # normalize the bounding boxes
        boxes = []
        for box in actual_boxes:
            boxes.append(normalize_box(box, width, height))

        # add as extra columns
        assert len(words) == len(boxes)
        example['words'] = words
        example['bbox'] = boxes
        return example
data.reset_index(drop=True, inplace=True)
dataset = Dataset.from_pandas(data)
updated_dataset = dataset.map(apply_ocr)

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

In [ ]:
data

,image_path,label
0,/content/drive/MyDrive/layout_dataset/invoive/...,invoive
1,/content/drive/MyDrive/layout_dataset/invoive/...,invoive
2,/content/drive/MyDrive/layout_dataset/invoive/...,invoive
3,/content/drive/MyDrive/layout_dataset/invoive/...,invoive
4,/content/drive/MyDrive/layout_dataset/invoive/...,invoive
5,/content/drive/MyDrive/layout_dataset/invoive/...,invoive
6,/content/drive/MyDrive/layout_dataset/invoive/...,invoive
7,/content/drive/MyDrive/layout_dataset/cv/Venu ...,cv
8,/content/drive/MyDrive/layout_dataset/cv/Venu ...,cv
9,/content/drive/MyDrive/layout_dataset/cv/kp/kp...,cv


In [ ]:
updated_dataset

Dataset({
    features: ['image_path', 'label', 'words', 'bbox'],
    num_rows: 18
})

In [ ]:
df = pd.DataFrame.from_dict(updated_dataset)
print(len(df["words"][0]))
print(df["words"][0])

178
['Tax', 'Invoice', 'Sold', 'By:', 'Jeeves', 'consumer', 'services', 'Private', 'Limited', ',', 'Ship-from', 'Address:', 'L-169,', '13th', 'Cross,', '5th', 'Main,,', 'Sector', '-', '6,', 'HSR', 'Layout,', 'BANGALORE,', 'KARNATAKA,', '560102,', 'IN-KA', 'L', 'GSTIN', '-', '29AABCJ9421C1ZP', 'Invoice', 'Number', '#', 'EAAAAB-05903550', ':', 'Order', 'ID:', 'OD120824647596905000', 'gi', 'To', 'Pammi', 'Venumadhav', '2-091,', '5th', 'ward,', 'shivasai', 'nagar,', 'khammam', 'road.', 'Kodad', '508206', 'Telangana', 'Order', 'Date:', '30-01-2021', 'Invoice', 'Date:', '30-01-2021', 'PAN:', 'AABCJ9421C', 'Ship', 'To', 'Pammi', 'Venumadhav', '2-091,', '5th', 'ward,', 'shivasai', 'nagar,', 'khammam', 'road.', '*Keep', 'this', 'invoice', 'and', 'manufacturer', 'box', 'for', 'Kodad', '508206', 'Telangana', 'warranty', 'purposes.', 'Phone:', 'xxxXxxXxxXxxXx', 'Phone:', 'xxxxxxxXxxx', 'Total', 'items:', '1', 'Product', 'Title', 'Qty', 'Gross', 'Discount', '=', 'Taxable', 'IGST', 'Total', '=', 'Am

In [ ]:
from transformers import LayoutLMTokenizer
import torch

tokenizer = LayoutLMTokenizer.from_pretrained("microsoft/layoutlm-base-uncased")

def encode_example(example, max_seq_length=512, pad_token_box=[0, 0, 0, 0]):
  words = example['words']
  normalized_word_boxes = example['bbox']

  assert len(words) == len(normalized_word_boxes)

  token_boxes = []
  for word, box in zip(words, normalized_word_boxes):
      word_tokens = tokenizer.tokenize(word)
      token_boxes.extend([box] * len(word_tokens))

  # Truncation of token_boxes
  special_tokens_count = 2
  if len(token_boxes) > max_seq_length - special_tokens_count:
      token_boxes = token_boxes[: (max_seq_length - special_tokens_count)]

  # add bounding boxes of cls + sep tokens
  token_boxes = [[0, 0, 0, 0]] + token_boxes + [[1000, 1000, 1000, 1000]]

  encoding = tokenizer(' '.join(words), padding='max_length', truncation=True)
  # Padding of token_boxes up the bounding boxes to the sequence length.
  input_ids = tokenizer(' '.join(words), truncation=True)["input_ids"]
  padding_length = max_seq_length - len(input_ids)
  token_boxes += [pad_token_box] * padding_length
  encoding['bbox'] = token_boxes
  encoding['label'] = label2idx[example['label']]

  assert len(encoding['input_ids']) == max_seq_length
  assert len(encoding['attention_mask']) == max_seq_length
  assert len(encoding['token_type_ids']) == max_seq_length
  assert len(encoding['bbox']) == max_seq_length

  return encoding

In [ ]:
from datasets import Features, Sequence, ClassLabel, Value, Array2D
# we need to define the features ourselves as the bbox of LayoutLM are an extra feature
features = Features({
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'attention_mask': Sequence(Value(dtype='int64')),
    'token_type_ids': Sequence(Value(dtype='int64')),
    'label': ClassLabel(names=['CV', 'Invoice','Bon','memo']),
    'image_path': Value(dtype='string'),
    'words': Sequence(feature=Value(dtype='string')),
})

encoded_dataset = updated_dataset.map(lambda example: encode_example(example),
                                      features=features)

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

In [ ]:
encoded_dataset.set_format(type='torch', columns=['input_ids', 'bbox', 'attention_mask', 'token_type_ids', 'label'])
dataloader = torch.utils.data.DataLoader(encoded_dataset, batch_size=1, shuffle=True)
batch = next(iter(dataloader))

In [ ]:
tokenizer.decode(batch['input_ids'][0].tolist())

"[CLS] description : natixis is a french corporate and investment bank created in november 2006 from the merger of the asset management and investment banking operations of natexis banques populaires and ixis. natixis provides financial data for the ‘ markets ’ section on the news channel. natixis in portugal is fully integrated in the global organization of natixis, a french multinational financial services firm specialized in asset & wealth management, corporate & investment banking, insurance and payments. a subsidiary of groupe bpce, natixis counts nearly 16. 000 employees across 38 countries. responsibilities : v involved to implement different python ocr libraries to detect the different image ocr. v experience in natural language processing, lexical analysis and morphological, syntactic analysis ( parsing ), semantic analysis, discourse integration, pragmatic analysis vy good experience in nltk library, file for processing, import required libraries, sentence tokenizing, word to

In [ ]:
from transformers import LayoutLMForSequenceClassification
import torch
from transformers import logging

logging.set_verbosity_warning()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LayoutLMForSequenceClassification.from_pretrained("microsoft/layoutlm-base-uncased", num_labels=len(label2idx))
model.to(device)

model.safetensors:   0%|          | 0.00/451M [00:00<?, ?B/s]

Some weights of LayoutLMForSequenceClassification were not initialized from the model checkpoint at microsoft/layoutlm-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LayoutLMForSequenceClassification(
  (layoutlm): LayoutLMModel(
    (embeddings): LayoutLMEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (x_position_embeddings): Embedding(1024, 768)
      (y_position_embeddings): Embedding(1024, 768)
      (h_position_embeddings): Embedding(1024, 768)
      (w_position_embeddings): Embedding(1024, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LayoutLMEncoder(
      (layer): ModuleList(
        (0-11): 12 x LayoutLMLayer(
          (attention): LayoutLMAttention(
            (self): LayoutLMSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=4e-5)

global_step = 0
num_train_epochs = 5
t_total = len(dataloader) * num_train_epochs # total number of training steps

#put the model in training mode
model.train()
for epoch in range(num_train_epochs):
  print("Epoch:", epoch)
  running_loss = 0.0
  correct = 0
  for batch in dataloader:
      input_ids = batch["input_ids"].to(device)
      bbox = batch["bbox"].to(device)
      attention_mask = batch["attention_mask"].to(device)
      token_type_ids = batch["token_type_ids"].to(device)
      labels = batch["label"].to(device)

      # forward pass
      outputs = model(input_ids=input_ids, bbox=bbox, attention_mask=attention_mask, token_type_ids=token_type_ids,
                      labels=labels)
      loss = outputs.loss

      running_loss += loss.item()
      predictions = outputs.logits.argmax(-1)
      correct += (predictions == labels).float().sum()

      # backward pass to get the gradients
      loss.backward()

      # update
      optimizer.step()
      optimizer.zero_grad()
      global_step += 1

  print("Loss:", running_loss / batch["input_ids"].shape[0])
  accuracy = 100 * correct / len(data)
  print("Training accuracy:", accuracy.item())

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch: 0
Loss: 14.354865282773972
Training accuracy: 61.11111068725586
Epoch: 1
Loss: 4.890822991728783
Training accuracy: 94.44444274902344
Epoch: 2
Loss: 0.8506583385169506
Training accuracy: 100.0
Epoch: 3
Loss: 0.36977594532072544
Training accuracy: 100.0
Epoch: 4
Loss: 0.19262800132855773
Training accuracy: 100.0


In [ ]:
model.save_pretrained('/content/drive/MyDrive/dataset/saved_model/')
tokenizer.save_pretrained('/content/drive/MyDrive/dataset/saved_model/')

('/content/drive/MyDrive/dataset/saved_model/tokenizer_config.json',
 '/content/drive/MyDrive/dataset/saved_model/special_tokens_map.json',
 '/content/drive/MyDrive/dataset/saved_model/vocab.txt',
 '/content/drive/MyDrive/dataset/saved_model/added_tokens.json')